In [1]:
import re
from pathlib import Path
import json
import csv
import numpy as np
import pandas as pd
from pathlib import Path
import json


log_dir = Path("../results/logs")
files = sorted(log_dir.glob("*.txt"))
pattern = re.compile(r'(\d+(?::\d+){1,2})\s*<00:00')

def time_str_to_seconds(ts: str) -> int:
    parts = [int(p) for p in ts.split(':')]
    if len(parts) == 2:
        minutes, seconds = parts
        return minutes * 60 + seconds
    elif len(parts) == 3:
        hours, minutes, seconds = parts
        return hours * 3600 + minutes * 60 + seconds
    else:
        raise ValueError(f"Unexpected time format: {ts}")

all_results = {}

for fp in files:
    text = fp.read_text(encoding="utf-8", errors="ignore")
    matches = pattern.findall(text)
    secs = []
    for t in matches:
        s = time_str_to_seconds(t)
        secs.append(s)
    all_results[fp.name] = {"time_strs": matches, "seconds": secs}

rows = []
for fname, data in all_results.items():
    secs = list(data.get("seconds", []))
    n = len(secs)
    start = max(0, n - 4)
    g1 = secs[start : start + 2]
    g2 = secs[start + 2 : start + 4]
    mean_g1 = float(np.nan) if len(g1) == 0 else float(np.mean(g1))
    mean_g2 = float(np.nan) if len(g2) == 0 else float(np.mean(g2))
    rows.append({
        "file": fname,
        "group1_vals": g1,
        "group2_vals": g2,
        "mean_group1_s": mean_g1,
        "mean_group2_s": mean_g2
    })

df = pd.DataFrame(rows).sort_values("file").reset_index(drop=True)
df['time'] = df.apply(lambda x: (x.mean_group1_s	 + x.mean_group2_s), axis=1)
df = df[['file', 'mean_group1_s', 'mean_group2_s', 'time']]
df.to_csv('time_b0.tsv', index=None, sep='\t')
